In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_excel("FX Daily Realized Covariance (Neural).xlsx", sheet_name= "Vol", index_col="Date",parse_dates=True)

In [3]:
df.head()

,AUD_JPY,EUR_USD,EUR_JPY,AUD_USD,GBP_USD,USD_CAD
Date,,,,,,
2005-01-02,0.002709,0.001914,0.002225,0.002490,0.001230,0.005083
2005-01-03,0.009263,0.008440,0.007271,0.010159,0.007134,0.007589
2005-01-04,0.009714,0.007693,0.007290,0.010561,0.006580,0.009502
2005-01-05,0.008100,0.006977,0.005789,0.009243,0.006430,0.007594
2005-01-06,0.007856,0.005759,0.006366,0.008229,0.005302,0.007154


In [4]:
train=df.iloc[:int(len(df)*0.7)]

In [5]:
test=df.iloc[int(len(df)*0.7)-22:]

In [6]:
df.iloc[int(len(df)*0.7)]

,2015-06-28
AUD_JPY,0.017456
EUR_USD,0.015948
EUR_JPY,0.025621
AUD_USD,0.008161
GBP_USD,0.004309
USD_CAD,0.002223


In [7]:
train=np.array(train)

In [8]:
train.shape

(3278, 6)

In [9]:
RV_d = []
for i in range(len(train)-1):
  RV_d.append(train[i].astype(float))

RV_w = []
for i in range(len(train)-5):
  RV_w.append(np.mean(train[i:i+5]).astype(float))

RV_m = []
for i in range(len(train)-22):
  RV_m.append(np.mean(train[i:i+22]).astype(float))

RV_d=RV_d[len(RV_d)-len(RV_m):]
RV_w=RV_w[len(RV_w)-len(RV_m):]

In [10]:
RV_D = np.zeros([len(RV_d),train.shape[1]])
for i in range(len(RV_d)):
  RV_D[i,:]=RV_d[i]

RV_W = np.zeros([len(RV_w),train.shape[1]])
for i in range(len(RV_w)):
  RV_W[i,:]=RV_w[i]

RV_M = np.zeros([len(RV_m),train.shape[1]])
for i in range(len(RV_m)):
  RV_M[i,:]=RV_m[i]

In [11]:
RV_actuals=train[22:,:]

In [12]:
RV_actuals.shape

(3256, 6)

In [13]:
test=np.array(test)

In [14]:
test.shape

(1428, 6)

In [15]:
  for i in range(len(test)-1):
    RV_d.append(test[i].astype(float))

  RV_w = []
  for i in range(len(test)-5):
    RV_w.append(np.mean(test[i:i+5]).astype(float))

  RV_m = []
  for i in range(len(test)-22):
    RV_m.append(np.mean(test[i:i+22]).astype(float))

  RV_d=RV_d[len(RV_d)-len(RV_m):]
  RV_w=RV_w[len(RV_w)-len(RV_m):]
  RV_D_test = np.zeros([len(RV_d),test.shape[1]])
  for i in range(len(RV_d)):
    RV_D_test[i,:]=RV_d[i]

  RV_W_test = np.zeros([len(RV_w),test.shape[1]])
  for i in range(len(RV_w)):
    RV_W_test[i,:]=RV_w[i]

  RV_M_test = np.zeros([len(RV_m),test.shape[1]])
  for i in range(len(RV_m)):
    RV_M_test[i,:]=RV_m[i]

In [16]:
forecasts=np.zeros([test.shape[0]-22,test.shape[1]])
for j in range(test.shape[1]):
  Data ={'Actual': RV_actuals[:,j],
  'D': RV_D[:,j],
  'W': RV_W[:,j],
  'M': RV_M[:,j]}
  dataframe = pd.DataFrame(data=Data)
  rv_scaled=dataframe
  rv_scaled = sm.add_constant(rv_scaled)
  rv_scaled['Actual']=rv_scaled['Actual'].astype(float)
  rv_scaled['D']=rv_scaled['D'].astype(float)
  rv_scaled['W']=rv_scaled['W'].astype(float)
  rv_scaled['M']=rv_scaled['M'].astype(float)
  X = rv_scaled.drop("Actual", axis = 1)
  y = rv_scaled["Actual"]
  results = sm.OLS(y, X).fit()
  Data ={'D': RV_D_test[:,j],
  'W': RV_W_test[:,j],
  'M': RV_M_test[:,j]}
  dataframe = pd.DataFrame(data=Data)
  rv_scaled=dataframe
  X_test = sm.add_constant(rv_scaled)
  forecasts[:,j]= results.predict(X_test)

In [17]:
df1=df.iloc[int(len(df)*0.7):]

In [18]:
df1.head()

,AUD_JPY,EUR_USD,EUR_JPY,AUD_USD,GBP_USD,USD_CAD
Date,,,,,,
2015-06-28,0.017456,0.015948,0.025621,0.008161,0.004309,0.002223
2015-06-29,0.010227,0.012577,0.013365,0.009544,0.006075,0.005885
2015-06-30,0.007963,0.008405,0.009202,0.008101,0.005222,0.006247
2015-07-01,0.006059,0.007375,0.007383,0.006609,0.004829,0.004640
2015-07-02,0.005837,0.007861,0.004529,0.009162,0.005909,0.006788


In [19]:
for i in range(len(df1.columns)):
  df1[df1.columns[i]]=forecasts[:,i]

<ipython-input-19-8a232fbfd1e1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[df1.columns[i]]=forecasts[:,i]


In [20]:
df1.to_excel("HAR Forecasts.xlsx")

In [21]:
test=df.iloc[int(len(df)*0.7):]
test=np.array(test)
RMSE=np.sqrt(np.sum((test-forecasts)**2)/(test.shape[0]*test.shape[1]))
MAE=(np.sum(abs(test-forecasts))/(test.shape[0]*test.shape[1]))
QLIKE=np.sum(test/forecasts-np.log(test/forecasts)-1)/(test.shape[0]*test.shape[1])
MAPE=np.sum(abs(test-forecasts)/(test))/(test.shape[0]*test.shape[1])
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

RMSE: 0.0025548829013217486 | QLIKE: 0.13579105550744694 | MAE: 0.0015573108091510494 | MAPE: 0.6372684639537968
